# Download Any BAM File 🌕

Any BAM file will do. I just want to see what aligned data looks like in the HCA.

First I'll set up the DSS client.

In [ ]:
import hca, hca.dss, json
client = hca.dss.DSSClient()

****
#### Now I want to find a bundle that has a BAM file in it.

The `client` has a method `get_file` that sounds very promising, but I need to get a UUID. I don't know what RFC4122 is, but hopefully I won't have to.

In [ ]:
help(client.get_file)

****
#### Searching for bundles

The `post_search` method accepts a `query`, and there's even an example query in the data-store repo's [readme](https://github.com/HumanCellAtlas/data-store/blob/master/README.md). The function signature doesn't quite match the example, but it should be easy to fix. 

In [ ]:
client.post_search(replica="aws", es_query={
    "query": {
        "bool": {
            "must": [{
                "match": {
                    "files.sample_json.donor.species": "Homo sapiens"
                }
            }, {
                "match": {
                    "files.assay_json.single_cell.method": "Fluidigm C1"
                }
            }, {
                "match": {
                    "files.sample_json.ncbi_biosample": "SAMN04303778"
                }
            }]
        }
    }
})

****
#### Well that didn't work!

But I can see how the results are structured. What if I just give it an empty query?

In [ ]:
search_response = client.post_search(replica="aws", es_query={})
search_response["total_hits"]

****
Okay great, many results. What does each result look like?

In [ ]:
search_response["results"][0]

____
Now I have an ID that I can work with, though it's an "fqid" not a "uuid", and it's for a bundle, not a file. I can try providing it to  `get_bundle`... 

In [ ]:
try:
    client.get_bundle(uuid=search_response["results"][0]["bundle_fqid"], replica="aws")
    print("Completed successfully!")
except Exception as e:
    # If this operation fails, let's print the error (without raising the exception)
    print("Oh no! There was an error.")
    print(e)

****
Hmmm, a `DSSException`; it appears that it couldn't find a bundle with that UUID. I suppose I can strip off the timestamp from the ID. (The UUID is the part of the FQID before the first `.`. The timestamp is everything after.)

In [ ]:
bundle_uuid, bundle_version = search_response['results'][0]['bundle_fqid'].split('.', maxsplit=1)
client.get_bundle(uuid=bundle_uuid, replica="aws")

****
This is very promising! No BAM files in this bundle, but I see the structure of a "bundle" and how I can work with it. Now, I just need to iterate over bundles until I find a BAM 

In [ ]:
for result in search_response["results"]:
    bundle_uuid = result["bundle_fqid"].split('.', maxsplit=1)[0]
    try:
        bundle_dict = client.get_bundle(uuid=bundle_uuid, replica="aws")
    except:
        # Sometimes, bundles that are deleted linger in the index.
        # It's no problem - we can just ignore it and move on.
        continue
    found_file = False
    for file_dict in bundle_dict["bundle"]["files"]:
        if file_dict["name"].endswith(".bam"):
            print("Name: {}, UUID: {}".format(file_dict["name"], file_dict["uuid"]))
            found_file = True
            break
    if found_file:
        break

What if I look for fastqs?

In [ ]:
for result in search_response["results"]:
    bundle_uuid = result["bundle_fqid"].split('.', maxsplit=1)[0]
    try:
        bundle_dict = client.get_bundle(uuid=bundle_uuid, replica="aws")
    except:
        continue
    found_file = False
    for file_dict in bundle_dict["bundle"]["files"]:
        if file_dict["name"].endswith(".fastq.gz"):
            print("Name: {}, UUID: {}".format(file_dict["name"], file_dict["uuid"]))
            found_file = True
            break
    if found_file:
        break

****

Don't see any results? That's because search results are paginated, so the `results` list only contains 100 of the ~2000 results. There's another RFC for the reading list there, but there's also the `post_search.iterate` method that should work.

In [ ]:
help(client.post_search)

In [ ]:
results = client.post_search.iterate(replica="aws", es_query={})
for result in results:
    bundle_uuid = result["bundle_fqid"].split('.', maxsplit=1)[0]
    try:
        bundle_dict = client.get_bundle(uuid=bundle_uuid, replica="aws")
    except:
        continue
    found_file = False
    for file_dict in bundle_dict["bundle"]["files"]:
        if file_dict["name"].endswith(".bam"):
            print("Name: {}, UUID: {}".format(file_dict["name"], file_dict["uuid"]))
            found_file = True
            break
    if found_file:
        break

In [ ]:
bam_file = client.get_file(uuid=file_dict['uuid'], replica="aws")
with open("Aligned.sortedByCoord.out.bam", "wb") as output_bam:
    output_bam.write(bam_file)

In [ ]:
import pysam
bam = pysam.AlignmentFile("Aligned.sortedByCoord.out.bam", "rb")
print(bam.header)